In [ ]:
import requests
import json
import os
import shutil
import time
import jionlp as jio
from openai import OpenAI
from http.client import RemoteDisconnected

# 获取当前工作目录
current_directory = "./A fengmaoyaosu/yaosu_2/paper_5"

# 获取当前文件夹下的所有PDF文件
pdf_files = [file for file in os.listdir(current_directory) if file.endswith('.pdf')]

# 保存所有拆分以后的文件夹地址
all_folder_paths = []

# 创建一个文件夹用于保存每个PDF文件，方便后面往知识库上传
for pdf_file in pdf_files:
    pdf_filename, _ = os.path.splitext(pdf_file)
    folder_name = pdf_filename + '_folder'
    folder_path = os.path.join(current_directory, folder_name)

    # 创建文件夹
    os.makedirs(folder_path, exist_ok=True)

    # 移动PDF文件到文件夹中
    shutil.move(os.path.join(current_directory, pdf_file), os.path.join(folder_path, pdf_file))

    all_folder_paths.append(folder_path)

# 获取 WSL 的 IP 地址
wsl_ip = os.popen("ip addr show eth0 | grep 'inet ' | awk '{print $2}' | cut -d/ -f1").read().strip()
print(f"WSL IP Address: {wsl_ip}")

# 初始化结果列表
all_results = []

# 定义结果文件的路径和文件名
output_directory = "./A fengmaoyaosu/yaosu_2/chouquresults_5"
os.makedirs(output_directory, exist_ok=True)

def save_results_to_file(file_name, results):
    output_file_path = os.path.join(output_directory, f"{file_name}.txt")
    with open(output_file_path, 'w', encoding='utf-8') as file:
        for item in results:
            file.write(f"Query: {item['query']}\n")
            file.write(f"Answer: {item['res']}\n")
            file.write("\n")
    print(f"Results for {file_name} have been saved to {output_file_path}")

def load_processed_files(output_directory):
    processed_files = set()
    for filename in os.listdir(output_directory):
        if filename.endswith('.txt'):
            processed_files.add(filename.replace('.txt', ''))
    return processed_files

# 已处理文件列表
processed_files = load_processed_files(output_directory)

for folder_path in all_folder_paths:
    # 获取当前文件名
    file_name = os.path.basename(folder_path).replace('_folder', '')
    
    # 跳过已处理的文件
    if file_name in processed_files:
        print(f"Skipping already processed file: {file_name}")
        continue

    print(f"Processing file: {file_name}")

    # 1.新建知识库，默认一篇PDF为一个知识库，
    url = f"http://{wsl_ip}:8777/api/local_doc_qa/new_knowledge_base"
    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "user_id": "zzp",  # 用户id，不需要改动
        "kb_name": file_name,  # 知识库名称，按需修改，暂时用的是论文名称
    }

    response = None
    while not response:
        try:
            response = requests.post(url, headers=headers, data=json.dumps(data, ensure_ascii=False))
            response.raise_for_status()
        except RemoteDisconnected:
            print("Remote server disconnected. Retrying...")
            response = None
            time.sleep(5)
        except (requests.ConnectionError, requests.Timeout):
            response = None
            print("Connection error or timeout occurred. Retrying...")
            time.sleep(5)

    print(f"New Knowledge Base Response: {response.json()}")

    current_kb_id = response.json()['data']["kb_id"]
    print(f"Knowledge Base ID: {current_kb_id}")

    # 2.上传文件
    url = f"http://{wsl_ip}:8777/api/local_doc_qa/upload_files"
    data = {
        "user_id": "zzp",
        "kb_id": current_kb_id,
        "mode": "soft"
    }

    files = []
    for root, dirs, file_names in os.walk(folder_path):
        for file_name in file_names:
            if file_name.endswith(".pdf"):  # 这里只上传后缀是pdf的文件，
                file_path = os.path.join(root, file_name)
                files.append(("files", open(file_path, "rb")))

    response = None
    while not response:
        try:
            response = requests.post(url, files=files, data=data)
            response.raise_for_status()
        except RemoteDisconnected:
            print("Remote server disconnected. Retrying...")
            response = None
            time.sleep(5)
        except (requests.ConnectionError, requests.Timeout):
            response = None
            print("Connection error or timeout occurred. Retrying...")
            time.sleep(5)

    print(f"Upload Files Response: {response.json()}")

    # 检查文件解析状态
    def check_file_status(file_id, current_kb_id, timeout=120):
        url = f"http://{wsl_ip}:8777/api/local_doc_qa/list_files"
        headers = {
            "Content-Type": "application/json"
        }
        data = {
            "user_id": "zzp",
            "kb_id": current_kb_id
        }
        start_time = time.time()
        while True:
            try:
                response = requests.post(url, headers=headers, data=json.dumps(data))
                response.raise_for_status()
                file_status = response.json()['data']['details'][0]['status']
                print(f"File Status: {file_status}")
                if file_status == 'green':
                    return True
                if time.time() - start_time > timeout:
                    print(f"File {file_id} processing timeout. Skipping...")
                    return False
                time.sleep(5)  # 等待 5 秒钟后再检查一次
            except RemoteDisconnected:
                print("Remote server disconnected. Retrying...")
                time.sleep(5)
            except (requests.ConnectionError, requests.Timeout):
                print("Connection error or timeout occurred. Retrying...")
                time.sleep(5)

    # 获取文件列表（POST）
    url = f"http://{wsl_ip}:8777/api/local_doc_qa/list_files"
    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "user_id": "zzp",
        "kb_id": current_kb_id
    }

    response = None
    while not response:
        try:
            response = requests.post(url, headers=headers, data=json.dumps(data))
            response.raise_for_status()
        except RemoteDisconnected:
            print("Remote server disconnected. Retrying...")
            response = None
            time.sleep(5)
        except (requests.ConnectionError, requests.Timeout):
            response = None
            print("Connection error or timeout occurred. Retrying...")
            time.sleep(5)

    print(f"List Files Response: {response.json()}")

    current_file_id = response.json()['data']['details'][0]['file_id']
    print(f"File ID: {current_file_id}")

    # 检查文件状态，直到文件解析完成或超时
    if not check_file_status(current_file_id, current_kb_id):
        print(f"Skipping file {file_name} due to processing timeout.")
        continue

    def rag__fengmao_qa(query, current_kb_id):
        url = f"http://{wsl_ip}:8777/api/local_doc_qa/local_doc_chat"
        headers = {
            'content-type': 'application/json'
        }
        data = {
            "user_id": "zzp",
            "kb_ids": [current_kb_id],
            "question": query
        }

        start_time = time.time()
        response = None
        while not response:
            try:
                response = requests.post(url=url, headers=headers, json=data, timeout=120, stream=False)
                response.raise_for_status()
                res = response.json()
                print("Response JSON:", json.dumps(res, indent=2, ensure_ascii=False))  # 打印响应内容以检查其结构

                if 'history' in res and len(res['history']) > 0:
                    # Assuming the answer is in the second item of the first list in 'history'
                    answers = res['history'][0][1]
                else:
                    answers = "Error: No history found in response."
            except RemoteDisconnected:
                print("Remote server disconnected. Retrying...")
                response = None
                time.sleep(5)
            except (requests.ConnectionError, requests.Timeout):
                print("Connection error or timeout occurred. Retrying...")
                response = None
                time.sleep(5)
            except (KeyError, IndexError) as e:
                print(f"Error: {e}")
                answers = "Error: The response structure is not as expected."

        return answers

    # 定义11个问题
    query_list = [
        "假设你是一位城市风貌研究领域专家，对城市风貌研判与塑造有持续深入的研究和贡献，请从文章中总结风貌的“物质形态”构成要素?",
        "请从文章中总结风貌的“自然环境”构成要素。“自然环境”是指“城市及其周边区域的自然地理特征和生态系统，包括所有未被人类开发或仅部分开发的自然要素”?",
        "请从文章中总结风貌的“城市空间格局”构成要素。“城市空间格局”是指“指整体性、宏观层面的，涉及整个城市或大区域的空间结构和布局”?",
        "请从文章中总结风貌的“道路结构”构成要素。“道路结构”是指“相对宏观层级的，涉及整个城市或区域内道路系统的整体组织和连接方式，关注的是道路网络的功能性和交通流动性?",
        "请从文章中总结风貌的“地块单元形态”构成要素。不同于城市空间格局， “地块单元形态”是指“指微观层面的，涉及具体地块的几何形态和空间分布?",
        "请从文章中总结风貌的“公共空间”构成要素。“公共空间”是指“城市中供公众日常使用和活动的开放空间的组织和形态特征。它涵盖了城市中所有对公众开放的场所，但我们这里不包括街道空间”?",
        "请从文章中总结风貌的“街道空间形态”构成要素。不同于道路结构，“街道空间形态”是是相对微观层级的，涉及具体街道及其周边区域的物理特征和设计，关注的是街道的使用体验和视觉效果?",
        "请从文章中总结风貌的“建筑形态”构成要素。“建筑形态”是指“城市中各类建筑物的外观和结构特征”?",
        "请从文章中总结风貌的“历史街区形态”构成要素。“历史街区形态”是指“具有历史和文化价值的城市区域的整体空间组织和布局特征，体现特定历史时期的城市风貌”?",
        "请从文章中总结风貌的“历史建筑形态”构成要素。“历史建筑形态”是指“具有历史和文化价值的单体建筑物的外观和结构特征”?",
        "请从文章中总结风貌的“历史遗址形态”构成要素。“历史遗址形态”是指“城市内具有考古和历史价值的遗址的空间形态和保存状况，代表特定历史时期的遗存特征?"
    ]

    # 获取每个问题的答案
    results = []
    for query in query_list:
        print(f"Query: {query}")
        answer = rag__fengmao_qa(query=query, current_kb_id=current_kb_id)
        print(f"Answer: {answer}")
        results.append({"query": query, "res": answer})

    # 将当前文件的结果保存到总结果列表
    all_results.append({
        "file_name": file_name,
        "results": results
    })

    # 保存当前文件的结果到文件
    save_results_to_file(file_name, results)

print(f"Results have been saved to {output_directory}")


In [ ]:
import os
from collections import defaultdict

def read_result_file(file_path):
    """
    读取单个结果文件，将其内容解析为问答对列表。
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read().strip().split('\n\n')
        result = []
        current_query = None
        current_answers = []
        for section in content:
            if section.startswith("Query:"):
                if current_query:
                    result.append({"query": current_query, "res": "\n\n".join(current_answers)})
                current_query = section
                current_answers = []
            elif section.startswith("Answer:") and current_query:
                current_answers.append(section)
        if current_query:
            result.append({"query": current_query, "res": "\n\n".join(current_answers)})
        return result
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return []

def load_all_results(result_directory):
    """
    从指定目录中加载所有结果文件，并将其内容解析为问答对列表。
    """
    try:
        result_files = [os.path.join(result_directory, file) for file in os.listdir(result_directory) if file.endswith('.txt')]
        all_results = {file: read_result_file(file) for file in result_files}
        print(f"Loaded results from {len(result_files)} files.")
        return all_results
    except Exception as e:
        print(f"Error loading results from directory {result_directory}: {e}")
        return {}

def merge_file_contents(content1, content2):
    """
    简单合并两个文件的内容。
    """
    return content1 + "\n\n" + content2

def merge_and_save_files(result_files, output_directory):
    """
    对指定文件进行两两合并，并输出合并后的文件。
    """
    os.makedirs(output_directory, exist_ok=True)
    merged_files = []
    
    result_file_paths = list(result_files.keys())
    for i in range(len(result_file_paths)):
        file1_path = result_file_paths[i]
        for j in range(i + 1, len(result_file_paths)):
            file2_path = result_file_paths[j]
            with open(file1_path, 'r', encoding='utf-8') as file1, open(file2_path, 'r', encoding='utf-8') as file2:
                content1 = file1.read().strip()
                content2 = file2.read().strip()
                merged_content = merge_file_contents(content1, content2)
                
                merged_file_name = f"merged_{i+1}_{j+1}.txt"
                merged_file_path = os.path.join(output_directory, merged_file_name)
                with open(merged_file_path, 'w', encoding='utf-8') as output_file:
                    output_file.write(merged_content)
                
                merged_files.append(merged_file_path)
    
    return merged_files

def parse_query_answer(content):
    """
    解析文档内容为Query和Answer对。
    """
    combined_results = defaultdict(list)
    lines = content.split('\n')
    current_query = None
    current_answer = []

    for line in lines:
        line = line.strip()
        if line.startswith("Query:"):
            if current_query:
                combined_results[current_query].append("\n".join(current_answer))
            current_query = line
            current_answer = []
        elif line.startswith("Answer:") and current_query:
            current_answer.append(line)
        elif current_query and line:
            current_answer[-1] += " " + line  # Append to the last answer line for multi-line answers
    if current_query:
        combined_results[current_query].append("\n".join(current_answer))
    
    return combined_results

def combine_answers_by_query(merged_file_path):
    """
    按问题合并答案，将所有相同问题的答案合并在一起。
    """
    combined_results = defaultdict(list)
    
    with open(merged_file_path, 'r', encoding='utf-8') as file:
        content = file.read().strip()
        combined_results.update(parse_query_answer(content))
    
    return combined_results

def save_combined_results(combined_results, output_file_path):
    """
    保存所有合并后的结果到一个文件中。
    """
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    with open(output_file_path, 'w', encoding='utf-8') as file:
        for query, answers in combined_results.items():
            combined_content = f"{query}\n\n" + "\n\n".join(answers) + "\n\n"
            file.write(combined_content)
    print(f"Combined results saved to {output_file_path}")

# 使用示例
result_directory = "./A fengmaoyaosu/Results/chouquresults"
merged_result_directory = "./A fengmaoyaosu/Results/MergedResults"
final_output_directory = "./A fengmaoyaosu/Results/FinalResults"

# 读取所有问答结果
result_files = load_all_results(result_directory)

# 两两合并文件内容并保存中间结果
merged_files = merge_and_save_files(result_files, merged_result_directory)
print(f"Merged files saved to {merged_result_directory}")

# 处理每个合并后的文件并保存最终结果
os.makedirs(final_output_directory, exist_ok=True)
for merged_file in merged_files:
    combined_results = combine_answers_by_query(merged_file)
    output_file_name = f"final_{os.path.basename(merged_file)}"
    output_file_path = os.path.join(final_output_directory, output_file_name)
    save_combined_results(combined_results, output_file_path)
print(f"Final combined results saved to {final_output_directory}")
